## Цель проекта:  
Проверить гипотезу о том, что пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы, и предоставить рекомендации для улучшения продукта.


## Задачи проекта:
- Загрузить и предобработать данные.
- Проверить наличие дубликатов и сравнить размеры групп.
- Провести статистический анализ для проверки гипотезы.
- Интерпретировать результаты и подготовить аналитическую записку с выводами и возможными причинами.
- Сформулировать рекомендации на основе полученных результатов.

## Описание данных

Источник данных: Yandex Knigi (файл: yandex_knigi_data.csv).

Структура файла:  
user_id: Уникальный идентификатор пользователя.  
city: Город пользователя (Москва или Санкт-Петербург).  
hours: Общее количество часов, проведенных пользователем за чтением и прослушиванием книг в приложении.

Импортируем библиотеки:

In [4]:
import pandas as pd
from scipy import stats
import numpy as np
import statsmodels.stats.api as sms
from math import ceil
import statsmodels.stats.proportion as proportions

Загрузим данные и выведем сводную информацию:

In [5]:
# Загрузка данных
df = pd.read_csv('https://code.s3.yandex.net/datasets/yandex_knigi_data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8784 non-null   int64  
 1   city        8784 non-null   object 
 2   puid        8784 non-null   int64  
 3   hours       8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 274.6+ KB


In [6]:
df.head()

,Unnamed: 0,city,puid,hours
0,0,Москва,9668,26.167776
1,1,Москва,16598,82.111217
2,2,Москва,80401,4.656906
3,3,Москва,140205,1.840556
4,4,Москва,248755,151.326434


Столбец 'Unnamed:0' дублирует индексы, его можно удалить

In [7]:
df = df.drop('Unnamed: 0', axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   city    8784 non-null   object 
 1   puid    8784 non-null   int64  
 2   hours   8784 non-null   float64
dtypes: float64(1), int64(1), object(1)
memory usage: 206.0+ KB


Снова выведем сводную иформацию:

In [8]:
df.head()

,city,puid,hours
0,Москва,9668,26.167776
1,Москва,16598,82.111217
2,Москва,80401,4.656906
3,Москва,140205,1.840556
4,Москва,248755,151.326434


Проверим наличие дубликатов в идентификаторах пользователей

In [9]:
# Проверка наличия дубликатов в идентификаторах пользователей
print("Количество дубликатов в 'puid':", df['puid'].duplicated().sum())

# Вывод первых нескольких строк для проверки
display(df.head())

Количество дубликатов в 'puid': 244


,city,puid,hours
0,Москва,9668,26.167776
1,Москва,16598,82.111217
2,Москва,80401,4.656906
3,Москва,140205,1.840556
4,Москва,248755,151.326434


В столбце 'puid' обнаружено 244 дубликата. Проанализируем их.

In [10]:
duplicates = df[df.duplicated(subset=['puid'], keep=False)] # keep=False  показывает все дубликаты
display(duplicates)

,city,puid,hours
35,Москва,2637041,10.317371
134,Москва,9979490,32.415573
145,Москва,10597984,42.931506
150,Москва,10815097,9.086655
187,Москва,13626259,21.104167
...,...,...,...
8771,Санкт-Петербург,1130000018516717,0.517778
8772,Санкт-Петербург,1130000018954257,33.583294
8773,Санкт-Петербург,1130000020425037,2.386944
8775,Санкт-Петербург,1130000023864516,14.384722


Удалим все дубликаты, оставив уникальные индефикаторы пользователей:

In [11]:
df.drop_duplicates(subset=['puid'], keep=False, inplace=True) #- удаляет все дубликаты (оставляет только уникальные puid).


In [12]:
# Проверка количества дубликатов после удаления
print("Количество дубликатов в 'puid' после удаления:", df['puid'].duplicated().sum())

# Проверка размера DataFrame после удаления
print("Размер DataFrame после удаления дубликатов:", df.shape)

Количество дубликатов в 'puid' после удаления: 0
Размер DataFrame после удаления дубликатов: (8296, 3)


Сравним размеры групп:

In [13]:
# Разделение данных по городам
moscow_data = df[df['city'] == 'Москва']['hours']
petersburg_data = df[df['city'] == 'Санкт-Петербург']['hours']

# Проверка размеров групп
print("Размер группы Москва:", len(moscow_data))
print("Размер группы Санкт-Петербург:", len(petersburg_data))

Размер группы Москва: 5990
Размер группы Санкт-Петербург: 2306


В выборке данных по Москве почти в 2.6 раза больше, чем по Санкт-Петербургу. Это существенная разница, которая может повлиять на результаты анализа. Чтобы уменьшить эту проблему, можно использовать сэмплирование (случайная выборка).

In [14]:
# Random sampling из Москвы, чтобы размер группы стал равен размеру группы Санкт-Петербурга
moscow_sampled = moscow_data.sample(n=len(petersburg_data), random_state=42)  # random_state для воспроизводимости

# Проверка размеров групп после сэмплирования
print("Размер группы Москва (после сэмплирования):", len(moscow_sampled))
print("Размер группы Санкт-Петербург:", len(petersburg_data))

# Теперь moscow_sampled и petersburg_data можно анализировать и сравнивать

Размер группы Москва (после сэмплирования): 2306
Размер группы Санкт-Петербург: 2306


Вывод размеров групп после сэмплирования подтверждает, что размеры теперь одинаковы (2306 и 2306), что и планировалось.

Выведем статистику и распределение групп:

In [15]:
# Вывод описательной статистики
print("\nОписательная статистика для Москвы:")
print(moscow_sampled.describe())
print("\nОписательная статистика для Санкт-Петербурга:")
print(petersburg_data.describe())


Описательная статистика для Москвы:
count    2306.000000
mean       10.947005
std        32.761708
min         0.000028
25%         0.056699
50%         0.864417
75%         5.984077
max       446.336651
Name: hours, dtype: float64

Описательная статистика для Санкт-Петербурга:
count    2306.000000
mean       11.264433
std        39.831755
min         0.000025
25%         0.060173
50%         0.875355
75%         6.138424
max       978.764775
Name: hours, dtype: float64


Более высокое стандартное отклонение и максимальное значение в Санкт-Петербурге указывают на то, что в этом городе разброс данных больше, и есть пользователи, проводящие значительно больше времени на платформе, чем в Москве.  
Близость медиан в обоих городах предполагает, что типичное время использования платформы схоже, несмотря на различия в выбросах.

## Проверка гипотезы в Python

Гипотеза звучит так: пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы. Попробуйте статистически это доказать, используя одностороннюю проверку гипотезы с двумя выборками:

- Нулевая гипотеза H₀: Средняя активность пользователей в часах в двух группах (Москва и Санкт-Петербург) не различается.

- Альтернативная гипотеза H₁: Средняя активность пользователей в Санкт-Петербурге больше, и это различие статистически значимо.

Учитывая наличие выбросов и неравномерное распределение можно использовать непараметрические тесты (например, тест Манна-Уитни), которые менее чувствительны к выбросам.

In [16]:
# Проверка гипотезы 
alpha = 0.05  # Уровень значимости

# Тест Манна-Уитни
results = stats.mannwhitneyu(petersburg_data, moscow_sampled, alternative='greater')

print('p-значение:', results.pvalue)

if results.pvalue < alpha:
    print("Отвергаем нулевую гипотезу")
    print("Cредняя активность пользователей в Санкт-Петербурге больше, и это различие статистически значимо.")
else:
    print("Не получилось отвергнуть нулевую гипотезу")
    print("Нет оснований утверждать, что средняя активность пользователей в Санкт-Петербурге больше, чем в Москве.")

p-значение: 0.4534772102258193
Не получилось отвергнуть нулевую гипотезу
Нет оснований утверждать, что средняя активность пользователей в Санкт-Петербурге больше, чем в Москве.


## Аналитическая записка по результатам сравнения активности пользователей в Москве и Санкт-Петербурге

Цель анализа: Проверить гипотезу о том, что пользователи из Санкт-Петербурга в среднем проводят больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы.

Метод исследования:
- Исходный набор данных показал существенную разницу в размерах групп, поэтому было применено случайное сэмплирование к данным по Москве, чтобы уравнять размеры групп (n=2306).
- Описательная статистика выявила более высокий разброс данных в Санкт-Петербурге (std = 39.83) по сравнению с Москвой (std = 32.76), а также более высокие максимальные значения. Медианные значения времени, проведенного в приложении, были сопоставимы в обоих городах.
- Для сравнения двух независимых выборок (активность пользователей в Москве и Санкт-Петербурге) был использован непараметрический тест Манна-Уитни, учитывая наличие выбросов и ненормальное распределение данных.
- При проверке гипотезы использовался односторонний тест, поскольку изначально выдвигалась гипотеза о том, что активность в Санкт-Петербурге больше, чем в Москве.
- Уровень статистической значимости (alpha) был установлен на уровне 0.05. 

Результаты теста:

- Полученное p-значение составило 0.453.

Поскольку p-значение (0.453) больше установленного уровня значимости alpha (0.05), у нас нет оснований отвергнуть нулевую гипотезу.
Это означает, что на основании имеющихся данных, нет оснований утверждать, что пользователи из Санкт-Петербурга проводят в среднем больше времени в приложении, чем пользователи из Москвы. 

- Возможные причины полученных результатов:

- Хотя размеры выборок были уравнены, возможно, что 2306 наблюдений недостаточно, чтобы выявить небольшие, но существующие различия между двумя группами. Больший размер выборки мог бы увеличить статистическую мощность теста и помочь обнаружить различия, если они есть.
- Возможно, что пользователи в Москве и Санкт-Петербурге действительно демонстрируют схожее поведение в отношении использования приложения. Различия в образе жизни, интересах и доступности контента могут быть не настолько значительными, чтобы привести к заметной разнице в активности.
- Большая стандартное отклонение (std) как для Москвы (32.76), так и для Санкт-Петербурга (39.83) говорит о значительной вариативности в данных. Некоторые пользователи проводят в приложении очень много времени, а другие – совсем мало. Это может затруднять выявление статистически значимых различий.




Теперь вам нужно проанализировать другие данные. Представьте, что к вам обратились представители интернет-магазина BitMotion Kit, в котором продаются геймифицированные товары для тех, кто ведёт здоровый образ жизни. У него есть своя целевая аудитория, даже появились хиты продаж: эспандер со счётчиком и напоминанием, так и подстольный велотренажёр с Bluetooth.

В будущем компания хочет расширить ассортимент товаров. Но перед этим нужно решить одну проблему. Интерфейс онлайн-магазина слишком сложен для пользователей — об этом говорят отзывы.

Чтобы привлечь новых клиентов и увеличить число продаж, владельцы магазина разработали новую версию сайта и протестировали его на части пользователей. По задумке, это решение доказуемо повысит количество пользователей, которые совершат покупку.

Ваша задача — провести оценку результатов A/B-теста. В вашем распоряжении:

* данные о действиях пользователей и распределении их на группы,

* техническое задание.

Оцените корректность проведения теста и проанализируйте его результаты.

## Опишите цели исследования.



Оценить, приведет ли внедрение новой версии сайта к увеличению количества пользователей, совершающих покупки, по сравнению с текущей версией сайта.

## Описание даных.

Параметры теста:
название теста: interface_eu_test;  
группы: А (контрольная), B (новый интерфейс).

Данные
https://code.s3.yandex.net/datasets/ab_test_participants.csv — таблица участников тестов.  
Структура файла:  
user_id — идентификатор пользователя;  
group — группа пользователя;  
ab_test — название теста;  
device — устройство, с которого происходила регистрация.

https://code.s3.yandex.net/datasets/ab_test_events.zip — архив с одним csv-файлом, в котором собраны события 2020 года;  
Структура файла:  
user_id — идентификатор пользователя;  
event_dt — дата и время события;  
event_name — тип события;  
details — дополнительные данные о событии.

Дополнительная информация по столбцу `details`  
Числовые значения:  
registration (регистрация) — стоимость привлечения клиента;  
purchase (покупка) — стоимость покупки.

## Загрузите данные, оцените их целостность.


In [17]:
participants = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_participants.csv')
events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_events.zip',
                     parse_dates=['event_dt'], low_memory=False)

Выведем общую информацию о данных (participants):

In [18]:
print("\nОбщая информация о participants:")
participants.info()
print("\nПервые 5 строк participants:")
display(participants.head())
print("\nПропуски в participants:")
print(participants.isnull().sum())
print("\nДубликаты в participants:")
print(participants.duplicated().sum())


Общая информация о participants:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB

Первые 5 строк participants:


,user_id,group,ab_test,device
0,0002CE61FF2C4011,B,interface_eu_test,Mac
1,001064FEAAB631A1,B,recommender_system_test,Android
2,001064FEAAB631A1,A,interface_eu_test,Android
3,0010A1C096941592,A,recommender_system_test,Android
4,001E72F50D1C48FA,A,interface_eu_test,Mac



Пропуски в participants:
user_id    0
group      0
ab_test    0
device     0
dtype: int64

Дубликаты в participants:
0


Таблица содержит 14525 строк и 4 столбца.  
Все столбцы имеют тип данных object (текст).  
В таблице нет пропущенных значений (все столбцы имеют 0 пропусков).  
В таблице нет полностью дублирующихся строк.

Выведем общую информацию о данных (events):

In [19]:
print("\nОбщая информация о events:")
events.info()
print("\nПервые 5 строк events:")
display(events.head())
print("\nПропуски в events:")
print(events.isnull().sum())
print("\nДубликаты в events:")
print(events.duplicated().sum())


Общая информация о events:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     787286 non-null  object        
 1   event_dt    787286 non-null  datetime64[ns]
 2   event_name  787286 non-null  object        
 3   details     249022 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.0+ MB

Первые 5 строк events:


,user_id,event_dt,event_name,details
0,GLOBAL,2020-12-01 00:00:00,End of Black Friday Ads Campaign,ZONE_CODE15
1,CCBE9E7E99F94A08,2020-12-01 00:00:11,registration,0.0
2,GLOBAL,2020-12-01 00:00:25,product_page,NaN
3,CCBE9E7E99F94A08,2020-12-01 00:00:33,login,NaN
4,CCBE9E7E99F94A08,2020-12-01 00:00:52,product_page,NaN



Пропуски в events:
user_id            0
event_dt           0
event_name         0
details       538264
dtype: int64

Дубликаты в events:
36318


Датафрейм events содержит 787286 строк и 4 столбца.  
Столбец details содержит значительное количество пропущенных значений (538264). Это говорит о том, что не для всех событий есть детали.  
В датафрейме 36318 дубликатов.

Удалим дублирующиеся строки:

In [20]:
events.drop_duplicates(inplace=True)
print("\nДубликаты после удаления:")
print(events.duplicated().sum())


Дубликаты после удаления:
0


## По таблице `ab_test_participants` оцените корректность проведения теста:

   3\.1 Выделите пользователей, участвующих в тесте, и проверьте:

   - соответствие требованиям технического задания,

   - равномерность распределения пользователей по группам теста,

   - отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

Выделим пользователей, участвующих в тесте:

In [21]:
test_name = 'interface_eu_test'
participants_test = participants[participants['ab_test'] == test_name].copy()

Проверим на соответствие требованиям технического задания (Проверка на группы A и B)

In [22]:
# Проверка корректности проведения теста
# Проверяем, что в тесте есть только группы А и B
print('\nГруппы в тесте:')
print(participants_test['group'].unique())


Группы в тесте:
['B' 'A']


В тесте присутствуют только группы A и B. Соответствует требованиям.

Проверим равномерность распределения пользователей по группам теста:

In [23]:
group_counts = participants_test['group'].value_counts()
print('\nРаспределение пользователей по группам:')
print(group_counts)


Распределение пользователей по группам:
group
B    5467
A    5383
Name: count, dtype: int64


Группа B: 5467 пользователей, Группа A: 5383 пользователей. Разница в количестве пользователей незначительна (84 пользователя). Распределение близко к равномерному. 

Проверим отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

In [24]:
# Находим все названия тестов, в которых участвуют пользователи из test_name
users_in_test = participants_test['user_id'].unique()
conflicting_tests = participants[participants['user_id'].isin(users_in_test)]['ab_test'].unique()
conflicting_tests = [test for test in conflicting_tests if test != test_name]

if len(conflicting_tests) > 0:
    print('\nОбнаружены пересечения с другими тестами:')
    print(conflicting_tests)
    # Более детальная проверка пересечений
    overlapping_users = participants[participants['user_id'].isin(users_in_test)].groupby('user_id')['ab_test'].nunique()
    overlapping_users = overlapping_users[overlapping_users > 1]
    print(f"Количество пользователей, участвующих в нескольких тестах: {len(overlapping_users)}")
else:
    print('\nПересечений с другими тестами не обнаружено.')


Обнаружены пересечения с другими тестами:
['recommender_system_test']
Количество пользователей, участвующих в нескольких тестах: 887


Тест interface_eu_test в целом проведен корректно (за исключением обнаруженных пересечений с другим тестом). Необходимо предпринять меры по устранению пересечений в будущих тестах, так как они приводят к искажению результатов.  
Удалим этих ползователей из анализа.

In [25]:
# Удаление пересекающихся пользователей 
users_to_exclude = overlapping_users.index
participants_test = participants_test[~participants_test['user_id'].isin(users_to_exclude)]
print(f"\nПосле удаления пересекающихся пользователей, в тесте осталось: {len(participants_test)} пользователей")


После удаления пересекающихся пользователей, в тесте осталось: 9963 пользователей


Снова проверим равномерность распределения пользователей по группам теста:

In [26]:
group_counts = participants_test['group'].value_counts()
print('\nРаспределение пользователей по группам:')
print(group_counts)


Распределение пользователей по группам:
group
B    5011
A    4952
Name: count, dtype: int64


После удаления пересекающихся пользователей, распределение по группам A и B осталось относительно сбалансированным.

Снова проверим отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

In [27]:
# Находим все названия тестов, в которых участвуют пользователи из test_name
users_in_test = participants_test['user_id'].unique()
conflicting_tests = participants[participants['user_id'].isin(users_in_test)]['ab_test'].unique()
conflicting_tests = [test for test in conflicting_tests if test != test_name]

if len(conflicting_tests) > 0:
    print('\nОбнаружены пересечения с другими тестами:')
    print(conflicting_tests)
    # Более детальная проверка пересечений
    overlapping_users = participants[participants['user_id'].isin(users_in_test)].groupby('user_id')['ab_test'].nunique()
    overlapping_users = overlapping_users[overlapping_users > 1]
    print(f"Количество пользователей, участвующих в нескольких тестах: {len(overlapping_users)}")
else:
    print('\nПересечений с другими тестами не обнаружено.')


Пересечений с другими тестами не обнаружено.


После удаления пересекающихся пользователей, не осталось участников, вовлеченных в несколько тестов.

3\.2 Проанализируйте данные о пользовательской активности по таблице `ab_test_events`:

- оставьте только события, связанные с участвующими в изучаемом тесте пользователями;

In [28]:
# Объединяем данные событий и участников теста
events_test = pd.merge(events, participants_test, on='user_id', how='inner')

- определите горизонт анализа: рассчитайте время (лайфтайм) совершения события пользователем после регистрации и оставьте только те события, которые были выполнены в течение первых семи дней с момента регистрации;

In [29]:
# Определяем дату регистрации для каждого пользователя
registration_dates = events_test[events_test['event_name'] == 'registration'].groupby('user_id')['event_dt'].min().reset_index()
registration_dates.rename(columns={'event_dt': 'registration_dt'}, inplace=True)

# Объединяем с основной таблицей
events_test = pd.merge(events_test, registration_dates, on='user_id', how='left')

# Рассчитываем лайфтайм 
# Создаем колонку с разницей между event_dt и registration_dt
events_test['lifetime_delta'] = events_test['event_dt'] - events_test['registration_dt']

# Фильтруем события, используя timedelta для сравнения
events_test = events_test[events_test['lifetime_delta'] <= pd.Timedelta(days=7)]

print(f"Количество событий после фильтрации по лайфтайму: {len(events_test)}")

# Проверка
print(events_test['lifetime_delta'].max())

Количество событий после фильтрации по лайфтайму: 58692
6 days 23:58:10


In [30]:
# Код ревьюера
(events_test.event_dt - events_test.registration_dt).max()

Timedelta('6 days 23:58:10')

Оцените достаточность выборки для получения статистически значимых результатов A/B-теста. Заданные параметры:

- базовый показатель конверсии — 30%,

- мощность теста — 80%,

- достоверность теста — 95%.

Оценим  достаточность выборки для A/B-теста


In [31]:
# Рассчитываем базовую конверсию (base_conversion)
base_conversion = 0.3  # Задана условиями задачи

# Определяем параметры теста
alpha = 0.05 # Доверительный интервал 95%
power = 0.8 # Мощность теста

# Подсчитываем количество пользователей в группах
group_size = events_test.groupby('group')['user_id'].nunique()

# Расчет минимального размера выборки
effect_size = sms.proportion_effectsize(base_conversion, base_conversion + 0.05)  # предполагаем минимальное изменение на 5%

required_n = sms.NormalIndPower().solve_power(
    effect_size=effect_size,
    power=power,
    alpha=alpha,
    ratio=1
)

required_n = ceil(required_n)

print(f"\nМинимальный размер выборки для каждой группы: {required_n}")

# Сравниваем размер выборки с минимальным требуемым
for group, size in group_size.items():
    if size >= required_n:
        print(f"В группе {group} размер выборки достаточен.")
    else:
        print(f"В группе {group} размер выборки недостаточен.  Необходимо хотя бы {required_n} пользователей.")

print('\nРаспределение пользователей по группам:')
print(group_size)


Минимальный размер выборки для каждой группы: 1376
В группе A размер выборки достаточен.
В группе B размер выборки достаточен.

Распределение пользователей по группам:
group
A    4952
B    5011
Name: user_id, dtype: int64


Исходя из предоставленной информации, размер выборки достаточен для проведения A/B-теста с заданными параметрами. В каждой группе (4952 и 5011)достаточно больше пользователей, чем требуется (1376).

- рассчитайте для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [32]:
# Рассчет конверсии по группам
purchase_counts = events_test[events_test['event_name'] == 'purchase'].groupby('group')['user_id'].nunique()
total_users = events_test.groupby('group')['user_id'].nunique()

conversion_rates = pd.DataFrame({
    'purchases': purchase_counts,
    'total_users': total_users
})

conversion_rates['conversion_rate'] = conversion_rates['purchases'] / conversion_rates['total_users']
conversion_rates = conversion_rates.reset_index()

print("\nКонверсия по группам:")
print(conversion_rates)


Конверсия по группам:
  group  purchases  total_users  conversion_rate
0     A       1377         4952         0.278069
1     B       1480         5011         0.295350


- сделайте предварительный общий вывод об изменении пользовательской активности в тестовой группе по сравнению с контрольной.

Конверсия в тестовой группе (B), где был применен обновленный дизайн, выше, чем в контрольной группе (A). Разница составляет 1.73 процентных пункта (29.54% - 27.81%). Это положительный результат, указывающий на потенциальное улучшение пользовательского опыта и увеличение продаж. Однако, улучшение меньше ожидаемого, так как разработчики ожидали прирост конверсии как минимум на 3 процентных пункта.



## Проведите оценку результатов A/B-тестирования:

- Проверьте изменение конверсии подходящим статистическим тестом, учитывая все этапы проверки гипотез.

Сформулируем гипотезы перед тестом:

- Нулевая гипотеза (H0): Конверсия в группе A равна конверсии в группе B. (pA = pB)

- Альтернативная гипотеза (H1): Конверсия в группе A меньше чем конверсия в группе B. (pA < pB) (Тест одностороний)

Наиболее подходящим тестом в данном случае, при сравнении конверсии между двумя группами, является z-тест для разности пропорций.



In [33]:
# Данные из расчета конверсии по группам
group_a_purchases = conversion_rates[conversion_rates['group'] == 'A']['purchases'].iloc[0]
group_a_total = conversion_rates[conversion_rates['group'] == 'A']['total_users'].iloc[0]
group_b_purchases = conversion_rates[conversion_rates['group'] == 'B']['purchases'].iloc[0]
group_b_total = conversion_rates[conversion_rates['group'] == 'B']['total_users'].iloc[0]

# Считаем z-статистику и p-value
z_stat, p_value = proportions.proportions_ztest(
    [group_a_purchases, group_b_purchases],
    [group_a_total, group_b_total],
    alternative='smaller'  # Указываем двустороннюю гипотезу
)

print(f"Z-статистика: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

alpha = 0.05  # Уровень значимости
if p_value < alpha:
    print("Отвергаем нулевую гипотезу: есть статистически значимая разница в конверсиях.")
else:
    print("Не отвергаем нулевую гипотезу: нет статистически значимой разницы в конверсиях.")



Z-статистика: -1.9070
P-value: 0.0283
Отвергаем нулевую гипотезу: есть статистически значимая разница в конверсиях.


- Опишите выводы по проведённой оценке результатов A/B-тестирования. Что можно сказать про результаты A/B-тестирования? Был ли достигнут ожидаемый эффект в изменении конверсии?

Выводы по результатам A/B-тестирования:

В ходе A/B-тестирования был проведен z-тест для разности пропорций для сравнения конверсии между группами A (контрольная) и B (с новым дизайном).

- Значение p-value = 0.0283, что меньше принятого уровня значимости (alpha = 0.05). Это позволяет отвергнуть нулевую гипотезу о равенстве конверсий в группах A и B.

- Наблюдается статистически значимое увеличение конверсии в группе B (обновленный дизайн) по сравнению с группой A.

- Конверсия в группе B составила 29.54%, а в группе A – 27.81%. Разница составляет 1.73 процентных пункта.

- Ожидаемый прирост конверсии составлял 3 процентных пункта. Фактический прирост меньше ожидаемого.

Заключение:

Результаты A/B-тестирования показывают статистически значимое улучшение конверсии при использовании обновленного дизайна (группа B). Хотя фактический прирост конверсии оказался меньше ожидаемого, положительный эффект подтвержден статистическими данными. Решение о внедрении нового дизайна может быть принято, учитывая статистическую значимость и положительное направление эффекта. Однако, для более полной оценки эффективности изменений, рекомендуется проанализировать и другие метрики, такие как средний чек и частота покупок.

